In [4]:
import numpy as np
from keras.preprocessing import sequence
import tensorflow as tf
import keras
import os

In [5]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [6]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [7]:
vocab = sorted(set(text))

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
    return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [8]:
print("Text:", text[:13])
print("Encoded:", text_as_int[:13])

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [9]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

In [10]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-05-06 18:36:11.796600: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-06 18:36:11.796691: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [15]:
for input_example_batch, target_example_batch in data.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

2022-05-06 18:36:12.195128: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [16]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 2.2176876e-04  7.0727040e-04 -4.2791520e-03 ...  2.3291917e-03
  -3.9425096e-03  1.4188811e-04]
 [ 4.3991795e-03 -1.6687263e-03 -7.9599197e-04 ...  3.2545424e-03
  -5.2271965e-03  2.5985839e-03]
 [ 4.6199923e-03 -7.2116404e-06  3.1628273e-03 ... -3.0035421e-03
  -1.3104080e-03  2.4936292e-03]
 ...
 [ 4.2266822e-03 -5.2450687e-05 -3.7534905e-03 ...  6.5197470e-03
  -1.6854563e-03  2.1408061e-03]
 [ 5.9012529e-03 -5.1851319e-03 -4.1676518e-03 ...  3.6051699e-03
   4.7858962e-04 -2.5539479e-04]
 [ 5.4455567e-03 -1.9071320e-03 -6.5619345e-03 ...  6.0726227e-03
  -3.1348742e-03 -4.9324398e-04]], shape=(100, 65), dtype=float32)


In [17]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 0.00022177  0.00070727 -0.00427915 -0.00546606  0.00217128 -0.00373018
 -0.0019904  -0.00065599  0.00042342 -0.00454957 -0.00117857 -0.00330887
 -0.0026486  -0.00374956 -0.00252339 -0.00145952 -0.00147351  0.00400613
 -0.00849762 -0.00274357 -0.00202941  0.00423283  0.00023066 -0.00210793
 -0.00124083  0.00433211  0.00125887 -0.00232495 -0.00030148 -0.00522529
  0.0018096  -0.00575584 -0.00075377  0.00128807  0.00294878 -0.00195249
  0.00453692  0.00561602  0.00114177  0.00074367  0.00297955 -0.00607606
 -0.00577155  0.00060396 -0.00450838  0.0044034  -0.00100021  0.00165113
 -0.00013097 -0.00285938 -0.00166074 -0.00311201 -0.00045654 -0.00206167
  0.00119113  0.0003034  -0.0089161   0.00120268 -0.00060476 -0.00353391
  0.00370178 -0.00038291  0.00232919 -0.00394251  0.00014189], shape=(65,), dtype=float32)


In [18]:
sampled_indices = tf.random.categorical(pred, num_samples=1)
sampled_indices = np.reshape(sampled_indices, (1,-1))[0]
predicted_chars = int_to_text(sampled_indices)
print(predicted_chars)

WpeZYSGdGLB EB
WFtgz!yizNEWwfPuxRHY,ejDgbqzTFsJy. RLS,;WpnKxSrfdYDOpHlXmRZ!UFU?OnnF-VTMhiA'HomwJSQ.I


In [19]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [20]:
model.compile(optimizer='adam', loss=loss)

In [21]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

In [22]:
# history = model.fit(data, epochs=10, callbacks=[checkpoint_callback])

In [23]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [24]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [25]:
def generate_text(model, start_string, num_generate=800):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []
    temperature = 1.0

    model.reset_states()

    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])
    
    return (start_string + ''.join(text_generated))

In [26]:
user_input = input("Enter your text: ")
print(generate_text(model, user_input, num_generate=1000))

oleg, I do come to beat warman.

BRUTUS:
But thou, so don; too means dapes-nain!
Have not Henry now to one Did of HaRD III:
Plause you, as might call you go on?
Thou shalt taken suca assall.

PETRUCHIO:
Save engresses in her kingdon's wifed had sad Angle, boy!
Imptheisk your brother's ladying; but where it your grace
As man, whom I am suck well danger my mind.
And bear Him I am, at heet thy depised, which ease your exinates.
Tend to your jow, drif! what sace my canst,
Stand was it impower'd with rich.

KING HENRY VI:
Fortent, an end?
If te two offend with thee faint a maid,
To false wrongs as young apparal untentable crown,
Thy dogs he do looked badised country and your fastiff;
For make wom not standle, but stif hinds even soul
And haves me Mantut is well; for Was to't undersonman'd, as he
Of ever I had for arone to glother, look up
The norther perfect of so, furtome of the duke.

ROMEO:
Ay, 'tis beloved her. Citizens!

CORIOLANUS:
I have upon your park.

Second Watchman:
I sut all.
G